In [13]:
## import library 
import os
import pandas as pd
import numpy as np
from ufunc import *

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [14]:
## set up authentication using services account 
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../bornok-004/sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../bornok-004/dti-ds-31329ac0651d.json"

In [15]:
# Load data from Google Biquery to local machine
project_id = 'dti-ds'
dataset_id = 'bornok_dataset_004'
table_id = 'data_test'
region = 'us-central1'
bucket_name = 'bornok_gcs_004'
blob_name = 'data/data_test.csv'

In [16]:
# Upload the model to Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('model_capstone/model.pkl')
    blob_model.upload_from_filename('model.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")

Uploading model succeeded


## Bulk & Sample prediction

In [17]:
sample_test_data = pd.DataFrame({
    'country' : ['ITA', 'PRT'],
    'market_segment' : ['Online TA', 'Offline TA/TO'],
    'previous_cancellations' : [0, 0],
    'booking_changes' : [0, 0],
    'deposit_type' : ['No Deposit', 'No Deposit'],
    'days_in_waiting_list' : [0, 215],
    'customer_type' : ["Transient", "Transient-Party"],
    'reserved_room_type' : ["A", "A"],
    'required_car_parking_spaces' : [0, 0],
    'total_of_special_requests' : [0, 0]
})

sample_test_data

,country,market_segment,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,reserved_room_type,required_car_parking_spaces,total_of_special_requests
0,ITA,Online TA,0,0,No Deposit,0,Transient,A,0,0
1,PRT,Offline TA/TO,0,0,No Deposit,215,Transient-Party,A,0,0


In [18]:
# load data from BQ
from google.cloud import bigquery
## using bigquery client
client = bigquery.Client(project=project_id)

# query
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
df = query_job.result().to_dataframe()

df.columns = sample_test_data.columns # added preprocessing for on cloud
df


# define threshold
threshold = 0.39

/home/bornouksyn497/miniconda3/envs/capstone/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [19]:
import pickle

# Load the model
with open ('model.pkl','rb') as f:
    loaded_model = pickle.load(f)

# Load the model transformer
with open ("model_transformer.pkl", "rb") as f2:
    loaded_model_transformer = pickle.load(f2)

In [20]:
# predict sample test data
x_preprocessed_sample = loaded_model_transformer[0].transform(sample_test_data)
y_pred_sample = loaded_model.predict_proba(x_preprocessed_sample)
y_pred_sample = y_pred_sample[:, -1]
y_pred_sample = np.where(y_pred_sample > threshold, 1, 0)
y_pred_sample

array([1, 1])

In [21]:
# integrating prediction into sample test data
sample_test_data["prediction"] = y_pred_sample
sample_test_data

,country,market_segment,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,reserved_room_type,required_car_parking_spaces,total_of_special_requests,prediction
0,ITA,Online TA,0,0,No Deposit,0,Transient,A,0,0,1
1,PRT,Offline TA/TO,0,0,No Deposit,215,Transient-Party,A,0,0,1


In [22]:
# predict bulk test data
x_preprocessed = loaded_model_transformer[0].transform(df)
y_pred_bulk = loaded_model.predict_proba(x_preprocessed)
y_pred_bulk = y_pred_bulk[:, -1]
y_pred_bulk = np.where(y_pred_bulk > threshold, 1, 0)
y_pred_bulk

array([1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0])

In [23]:
# bulk data with prediction
df["prediction"] = y_pred_bulk
df

,country,market_segment,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,reserved_room_type,required_car_parking_spaces,total_of_special_requests,prediction
0,PRT,Direct,0,0,No Deposit,0,Transient,A,0,0,1
1,PRT,Direct,0,0,No Deposit,0,Transient,A,1,0,0
2,PRT,Direct,0,1,No Deposit,0,Transient,H,0,0,0
3,PRT,Direct,0,1,No Deposit,0,Transient,C,0,0,1
4,PRT,Groups,0,0,Non Refund,49,Transient,A,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
95,GBR,Offline TA/TO,0,0,No Deposit,0,Contract,D,0,2,0
96,FRA,Offline TA/TO,0,1,No Deposit,0,Transient-Party,A,0,2,0
97,PRT,Online TA,0,0,No Deposit,0,Transient,A,0,3,1
98,CHE,Online TA,0,0,No Deposit,0,Transient,D,0,4,1


In [24]:
from google.cloud import bigquery

table_id = 'test_data_with_prediction'

# Construct a bigquery client object
client = bigquery.Client()

# define the full table id
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

df.columns = [''.join(i.split(' ')) for i in df.columns]

job = client.load_table_from_dataframe(df,table_full_id)

job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

Loaded 100 rows into dti-ds.bornok_dataset_004.test_data_with_prediction
